In [1]:
import numpy as np
import pandas as pd
from __future__ import division
import time
from random import shuffle
import xmltodict
import json
import collections
from collections import Counter

import nltk
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import *
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix

from gensim.models import LdaModel
from gensim.corpora import Dictionary

# view all columns of pandas df
pd.set_option('display.max_columns', None)

# nltk.download('wordnet')      # download wordnet if it's not already downloaded
# nltk.download('averaged_perceptron_tagger')

with open ("dreambank-public.xml") as f:
    doc = xmltodict.parse(f.read())

def convert(data):
    if isinstance(data, basestring):
        return str(data)
    elif isinstance(data, collections.Mapping):
        return dict(map(convert, data.iteritems()))
    elif isinstance(data, collections.Iterable):
        return type(data)(map(convert, data))
    else:
        return data

def left(s, amount):
    return s[:amount]

def right(s, amount):
    return s[-amount:]

def mid(s, offset, amount):
    return s[offset:offset+amount]

# Exploratory Analysis

First we want to print the data to see which fields we are given and how the data looks.

In [2]:
#==================================================================
# Cell Purpose: EDA
#               print the fields from the dreambank collection
# Flags: None in cell
#==================================================================

for dreamer in doc['dreambank']['collection']:
    print dreamer['name'] + ' (' + str(len(dreamer['dream'][:])) + ' dreams)' 
    print '  ID: ' + dreamer['id']
    print '  type: ' + dreamer['type']
    print '  sex: ' + dreamer['sex']
    print '  age: ' + dreamer['age']
    
    try:
        print '  time: ' + dreamer['time']
    except:
        pass
    
    print '  sample dream: ' 
    
    odict = dreamer['dream'][0]
    for key, value in odict.items():
        if convert(key) == 'report':
            print '    report: ' + left(convert(value), 200) + '...'
        else:
            print '    ' + convert(key) + ': ' + str(convert(value))
        
    print '\n'

Alta: a detailed dreamer (422 dreams)
  ID: alta
  type: series
  sex: F
  age: A
  time: 1985-1997
  sample dream: 
    number: 1
    date: 1957
    report: The one at the Meads's house, where it's bigger inside than out; there's a European village just inside, with a cobblestone street and a Pied-Piper sort of man with curly hair, he can do things like j...


Angie: age 18 & 20 (48 dreams)
  ID: angie
  type: series
  sex: F
  age: Y
  time: 1996
  sample dream: 
    number: 1-01
    date: 1996-04-03
    report: My memory of this dream is vague. I think the setting is on a college campus. I'm in a cafe and two elderly ladies walk in and start talking to me about a university that a guy I am dating got into fo...


Arlie: a middle-aged woman (212 dreams)
  ID: arlie
  type: series
  sex: F
  age: A
  time: 1992-1998
  sample dream: 
    number: 1
    date: 10/14/92
    report: I am in an office in the town next to the town I grew up in. Everyone is taking a rest. I have to go to the b

In [3]:
#==================================================================
# Cell Purpose: EDA
#               Print the number of dreams per deramer
#               Print the w266 ID 
#                   Note:  w266ID starts counting at 1, not 0
# Flags: None in cell
#==================================================================
print '---Dream collections from individuals---' + '\n'
MultIDs = ['b', 'madeline1-hs', 'madeline2-dorms', 'madeline3-offcampus', 'phil1', 'phil2', 'vietnam_vet']
NumberOfSeries = 1

for dreamer in doc['dreambank']['collection']:
    if dreamer['type'] == 'series':
        print '{' + dreamer['id']  + '} ' + dreamer['name'] + ' (' + str(len(dreamer['dream'][:])) + ' dreams)' + ' [' + dreamer['sex'] + ']'    
        dreamer['w266ID'] = NumberOfSeries
        
        # Assign a dreamer ID that groups the same dreamers together 
        # and skips the dream collections of multiple dreamers
        print 'w266 ID: ' + str(dreamer['w266ID'])
        if dreamer['id'] not in MultIDs:
            print '\n'
            NumberOfSeries += 1
    else:
        dreamer['w266ID'] = 0
        
print "Total Number of individuals to test vs. 'others': " + str(NumberOfSeries - 1)

---Dream collections from individuals---

{alta} Alta: a detailed dreamer (422 dreams) [F]
w266 ID: 1


{angie} Angie: age 18 & 20 (48 dreams) [F]
w266 ID: 2


{arlie} Arlie: a middle-aged woman (212 dreams) [F]
w266 ID: 3


{b} Barb Sanders (3116 dreams) [F]
w266 ID: 4
{b2} Barb Sanders #2 (1138 dreams) [F]
w266 ID: 4


{bosnak} Robert Bosnak: A dream analyst (53 dreams) [M]
w266 ID: 5


{chris} Chris: a transvestite (100 dreams) [M]
w266 ID: 6


{chuck} Chuck: a physical scientist (75 dreams) [M]
w266 ID: 7


{dahlia} Dahlia: concerns with appearance (24 dreams) [F]
w266 ID: 8


{david} David: teenage dreams (166 dreams) [M]
w266 ID: 9


{dorothea} Dorothea: 53 years of dreams (900 dreams) [F]
w266 ID: 10


{ed} Ed: dreams of his late wife (143 dreams) [M]
w266 ID: 11


{edna} Edna: a blind woman (19 dreams) [F]
w266 ID: 12


{elizabeth} Elizabeth: a woman in her 40s (1707 dreams) [F]
w266 ID: 13


{emma} Emma: 48 years of dreams (1521 dreams) [F]
w266 ID: 14


{emmas_husband} Emma's

In [4]:
#==================================================================
# Cell Purpose: EDA
#               Sum total of all dreams
# Flags: None in cell
#==================================================================
DreamNum = 0

for dreamer in doc['dreambank']['collection']:
    for odict in dreamer['dream']:
        for key, value in odict.items():            
            if convert(key) == 'report':
                DreamNum += 1

print "Total Dreams: " + str(DreamNum)

Total Dreams: 26000


# EDA - Reduce To Noun-Only And Lemmatize

For our topic modeling, we will want to lemmatize the corpus and reduce to nouns-only. However, before we get to topic modeling, it will be helpful to test out the lemmatization and noun-only reduction techniques.

In [5]:
#==================================================================
# Cell Purpose: EDA
#               Explore Lemmatiziation
#               Explore Reduction to Nouns Only
# Libraries: WordNetLemmatizer()
# Flags: None in cell
#==================================================================

# reduce to noun-only and lemmatize
lmtzr = WordNetLemmatizer()

for dreamer in doc['dreambank']['collection']:
    print dreamer['name'] + ' (' + str(len(dreamer['dream'][:])) + ' dreams)' + ' [' + dreamer['sex'] + ']'
    
    print '  noun-only sample dream: ' 
    
    odict = dreamer['dream'][0]
    
    # (1) tokenize dream text
    # (2) tag with PoS
    # (3) reduce to noun-only
    # (4) lemmatize nouns
    # (5) get frequency counts of the lemmatized nouns
    for key, value in odict.items():
        if convert(key) == 'report':
            text = nltk.tokenize.word_tokenize(value)
            pos = nltk.pos_tag(text)
            noun_only = [w[0] for w in pos if w[1].startswith('N')]
            lmtz_noun_only = [lmtzr.lemmatize(word) for word in noun_only]
            counts = Counter(lmtz_noun_only)
            print counts
        
    print '\n'

Alta: a detailed dreamer (422 dreams) [F]
  noun-only sample dream: 
Counter({u'house': 3, u'hair': 2, u'[': 2, u'room': 2, u'bridge': 1, u'set': 1, u'creek': 1, u'people': 1, u'stove': 1, u'one': 1, u'street': 1, u'village': 1, u'corner': 1, u'blonde': 1, u'string': 1, u'pageboy': 1, u'balloon': 1, u'juggle': 1, u'couple': 1, u'sort': 1, u'hallway': 1, u'woman': 1, u'Meads': 1, u'Inside': 1, u'cobblestone': 1, u']': 1, u'man': 1, u'drive': 1, u'round': 1, u'thing': 1, u'aunt': 1, u'side': 1})


Angie: age 18 & 20 (48 dreams) [F]
  noun-only sample dream: 
Counter({u'dream': 3, u'school': 2, u'guy': 2, u'lady': 2, u'hospital': 1, u'information': 1, u'feeling': 1, u'art': 1, u'orientation': 1, u'university': 1, u'cafe': 1, u'setting': 1, u'college': 1, u'memory': 1, u'law': 1, u'campus': 1})


Arlie: a middle-aged woman (212 dreams) [F]
  noun-only sample dream: 
Counter({u'town': 2, u'toilet': 2, u'bathroom': 1, u'Everyone': 1, u'office': 1, u'rest': 1})


Barb Sanders (3116 dreams) [F

  noun-only sample dream: 
Counter({u'style': 1, u'color': 1, u'account': 1, u'housemate': 1, u'lot': 1, u'windbreaker': 1, u'chair': 1, u'quality': 1, u'display': 1})


Mark: a young boy (23 dreams) [M]
  noun-only sample dream: 
Counter({u'car': 4, u'house': 4, u'hand': 3, u'Penney': 2, u'JC': 2, u'drink': 2, u'boy': 1, u'story': 1, u'bicycle': 1, u'mom': 1, u'park': 1, u'school': 1, u'bed': 1, u'matter': 1, u'teacher': 1, u'lot': 1, u'girl': 1, u'John': 1, u'.and': 1, u'guy': 1, u'fort': 1})


Melissa: a young girl (89 dreams) [F]
  noun-only sample dream: 
Counter({u'house': 2, u'Shirkan': 2, u'tiger': 1, u'microphone': 1, u'radio': 1})


Merri: an artist (315 dreams) [F]
  noun-only sample dream: 
Counter({u'window': 4, u'Chinatown': 3, u'sister': 3, u'corner': 2, u'girl': 2, u'picture': 2, u'time': 2, u'lantern': 2, u'Canal': 1, u'scam': 1, u'show': 1, u'South': 1, u'house': 1, u'St.': 1, u'desk': 1, u'yellow': 1, u'hair': 1, u'paper': 1, u'comma': 1, u'experience': 1, u'Carlos':

Counter({u'home': 5, u'snake': 3, u'dream': 3, u'tree': 2, u'year': 2, u'forest': 2, u'way': 2, u'friend': 2, u'Stevenson': 2, u'UCSC': 2, u'side': 2, u'gathering': 1, u'right': 1, u'point': 1, u'walk': 1, u'provost': 1, u'something': 1, u'Linda': 1, u'LA': 1, u'Cruz': 1, u'pile': 1, u'party': 1, u'type': 1, u'banana': 1, u'door': 1, u'Santa': 1, u'nothing': 1, u'beginning': 1, u'hall': 1, u'school': 1, u'Clara': 1, u'thing': 1, u'College': 1, u'time': 1, u'UC': 1, u'left': 1})


Vickie: a 10-year-old girl (35 dreams) [F]
  noun-only sample dream: 
Counter({u'cookie': 6, u'mom': 3, u'grocery': 2, u'ride': 2, u'doll': 2, u'guy': 2, u'store': 2, u'counter': 1, u'family': 1, u'area': 1, u'cooky': 1, u'dream': 1, u'grasshopper': 1, u'spider': 1, u'someplace': 1, u'something': 1, u'housing': 1, u'lady': 1, u'university': 1, u'kid': 1})


Vietnam Vet: 1970-2008 war dreams (98 dreams) [M]
  noun-only sample dream: 
Counter({u'hammer': 3, u'boot': 3, u'.45': 2, u'Cambodia': 2, u'claymore': 1, 

# Create Dataframe

In [6]:
#==================================================================
# Cell Purpose:   Wrangle Data into Dataframe
# Data Structure: Panda's DataFrame
# Flags:          None
#==================================================================

# Starting point for Columns/Variables in DataFrame
ID = 0
W266ID = 0
DreamBankID = ''
DreamNumber = ''
Name = ''
Sex = ''
Dream = ''
HasDream = 0

df = pd.DataFrame(columns=["ID", "W266ID", "DreamBankID", "DreamNumber", "Name", "Sex", "Dream"])

for dreamer in doc['dreambank']['collection']:
    Name = dreamer['name'] 
    DreamBankID = dreamer['id']
    W266ID = int(dreamer['w266ID'])
    Sex = dreamer['sex']
    
    for odict in dreamer['dream']:
        HasDream = 0
        for key, value in odict.items():
            if convert(key) == 'report':
                Dream = convert(value)
                HasDream = 1
            if convert(key) == 'number':
                DreamNumber = convert(value)
        
        if HasDream == 1:
            ID += 1
            df = df.append({
                "ID": ID,
                "W266ID": W266ID,
                "DreamBankID": DreamBankID,
                "DreamNumber": DreamNumber,
                "Name": Name,
                "Sex": Sex,
                "Dream": Dream
                }, ignore_index=True)        

# change W266ID from float to integer
df.W266ID = df.W266ID.astype(int)

# print df
print "Total Dreams: " + str(ID)
print "\n"

df.head()

Total Dreams: 26000




,ID,W266ID,DreamBankID,DreamNumber,Name,Sex,Dream
0,1,1,alta,1,Alta: a detailed dreamer,F,"The one at the Meads's house, where it's bigge..."
1,2,1,alta,2,Alta: a detailed dreamer,F,I'm at a family reunion in a large fine house ...
2,3,1,alta,3,Alta: a detailed dreamer,F,I watch a plane fly past and shortly realize i...
3,4,1,alta,4,Alta: a detailed dreamer,F,Me pulling the green leaves and berries off so...
4,5,1,alta,5,Alta: a detailed dreamer,F,I'm in a room that reminds me of (but definite...


In [7]:
print 'Males: ' + str(len(df[df['Sex']=='M']))
print 'Females: ' + str(len(df[df['Sex']=='F']))
print 'Total: ' + str(len(df))

Males: 7813
Females: 18187
Total: 26000


# Modeling

We want to create a different model for each dreamer (i.e. a classifier identifying one vs all-others for each dreamer). This will allow us to identify the most predictive words in identifying each dreamer.

In [8]:
#==================================================================
# Cell Purpose:  randomly shuffle dataframe
#               set seed for consistency while running
#==================================================================
np.random.seed(0)
df = df.sample(frac=1).reset_index(drop=True)

# add 41 flag columns denoting dreamers (one col per dreamer) 
dreamer_flag = pd.get_dummies(df['W266ID'], prefix='Dreamer')
df = pd.concat([df, dreamer_flag], axis=1)

# create vocab from all dreams
dreams_flat = df['Dream'].values.flatten().tolist()
dreams_list = " ".join(dreams_flat)
vocab = list(set(nltk.tokenize.word_tokenize(dreams_list)))

In [9]:
#==================================================================
# Cell Purpose: Function to split the df into dev, test, train sets
#==================================================================

def split_data(df, W266ID, train=0.6):        
    # column for "our" dreamer
    dreamer_label = 'Dreamer_' + str(W266ID)
    
    # make 60/40 split of train/test
    # test will be evenly split between dev and test in the next step
    num_train = int(len(df) * train)
    num_test = int(len(df) * (1-train)) 

    train_data, train_labels = df['Dream'][:num_train], df[dreamer_label][:num_train]
    dev_data, dev_labels = df['Dream'][-num_test : -num_test//2], df[dreamer_label][-num_test : -num_test//2] 
    test_data, test_labels = df['Dream'][-num_test//2:], df[dreamer_label][-num_test//2:]

    return train_data, train_labels, dev_data, dev_labels, test_data, test_labels

In [10]:
#==================================================================
# Cell Purpose:  EDA
#                Explore dreamer 1 (Alta's) data by creating 1 versus others
#                Create BOW from CountVectorizer
#                Create TFIDF 
#                Run Logistic Regression
# Flag:
#      SHOW_DREAMER_ONE = True 
#           Cell will split the dataset into training, dev, and test sets
#      SHOW_DERAMER_ONE = False 
#           Nothing in the cell will execute
#=============================================================
SHOW_DREAMER_ONE = False 

if SHOW_DREAMER_ONE:
    # # split data for dreamer1 - Alta
    train_data, train_labels, dev_data, dev_labels, test_data, test_labels = split_data(df, W266ID=1)

    ## Create a Bag-of-Words Vectorizer
    vec = CountVectorizer(vocabulary=vocab)
    vec_bow_train_data = vec.fit_transform(train_data)
    vec_bow_dev_data = vec.transform(dev_data)  

    # # Create a Tfidf Vectorizer
    vec_tfidf = TfidfVectorizer(stop_words='english')
    vec_tfidf_train_data = vec_tfidf.fit_transform(train_data)
    vec_tfidf_dev_data   = vec_tfidf.transform(dev_data)

    best_lr_score = 0

    # ## Logistic reg
    for c in (0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 500):
        bow_log = LogisticRegression(C = c)
        bow_log.fit(vec_bow_train_data, train_labels)
    
        tfidf_logit_model = LogisticRegression(C=c)
        tfidf_logit_model.fit(vec_tfidf_train_data, train_labels)
    
        f1_bow_lr_score = metrics.f1_score(dev_labels, bow_log.predict(vec_bow_dev_data), average='macro')    
        f1_tfidf_lr_score = metrics.f1_score(dev_labels, tfidf_logit_model.predict(vec_tfidf_dev_data), average='macro') 

        print 'Logistic Reg:\t C=%3.4f\t BOW: F1-score=%3.3f\t TFIDF: F1-score=%3.3f' % (c, f1_bow_lr_score, f1_tfidf_lr_score)

        if f1_bow_lr_score > best_lr_score:
            best_lr_score = f1_bow_lr_score
            best_C = c 
            vectorizer = 'BOW'
        
        if f1_tfidf_lr_score > best_lr_score:
            best_lr_score = f1_tfidf_lr_score
            best_C = c
            vectorizer = 'TFIDF'

    print ''
    print 'Best model:\t C=%3.4f\t vectorizer = %s\t F1-score=%3.3f' % (best_C, vectorizer, best_lr_score)

## Run a model for each dreamer

We need to run a separate model for each dreamer. The models will predict if a dream comes from that dreamer or from "all-others". Since bag-of-words was working best in the above models, we will continue to use that for our baseline.

In [11]:
#==================================================================
# Cell Purpose:  Run Logisitic Regression for each dreamer
#                This creates a logistic Regression for Each Dreamer
#                with a "one" versus "rest" methodology
#==================================================================
PROGRESSION_PRINTOUTS_ON = True

cell_start = time.time()

models = {}

for i in range(0,41):
    if PROGRESSION_PRINTOUTS_ON:
        print "Progression: Logistic Regression for dreamer ", i
    # split data
    train_data, train_labels, dev_data, dev_labels, test_data, test_labels = split_data(df, W266ID=i)

    # Create a Bag-of-Words Vectorizer
    vec_bow = CountVectorizer(vocabulary=vocab)
    vec_bow_train_data = vec_bow.fit_transform(train_data)
    vec_bow_dev_data = vec_bow.transform(dev_data)  
    
    # Create a TF Vectorizer
    vec_tf = TfidfTransformer(use_idf=False, smooth_idf=False)
    vec_tf_train_data = vec_tf.fit_transform(vec_bow_train_data)
    vec_tf_dev_data   = vec_tf.transform(vec_bow_dev_data)     

    # Create a TF-IDF Vectorizer
    vec_tfidf = TfidfVectorizer(stop_words='english')
    vec_tfidf_train_data = vec_tfidf.fit_transform(train_data)
    vec_tfidf_dev_data   = vec_tfidf.transform(dev_data)    

    ## Logistic reg
    bow_log = LogisticRegression(C = 100)
    bow_log.fit(vec_bow_train_data, train_labels)

    tf_log = LogisticRegression(C = 100)
    tf_log.fit(vec_tf_train_data, train_labels)
    
    tfidf_log = LogisticRegression(C = 100)
    tfidf_log.fit(vec_tfidf_train_data, train_labels)    
    
    # score models
    f1_bow = metrics.f1_score(dev_labels, bow_log.predict(vec_bow_dev_data), average='macro')
    f1_tf = metrics.f1_score(dev_labels, tf_log.predict(vec_tf_dev_data), average='macro')    
    f1_tfidf = metrics.f1_score(dev_labels, tfidf_log.predict(vec_tfidf_dev_data), average='macro')

    # find the most-predictive features
    ### we're not using the weights currently, but might be useful/interesting later ###    
    best_feature_positions_bow = bow_log.coef_.argsort()[0][-5::]
    best_feature_weights_bow = bow_log.coef_[0][best_feature_positions_bow.astype(int)]
    
    best_feature_positions_tf = tf_log.coef_.argsort()[0][-5::]
    best_feature_weights_tf = tf_log.coef_[0][best_feature_positions_tf.astype(int)]  
    
    best_feature_positions_tfidf = tfidf_log.coef_.argsort()[0][-5::]
    best_feature_weights_tfidf = tfidf_log.coef_[0][best_feature_positions_tfidf.astype(int)]    

    # get word labels for our features
    bow_best_words = []
    for ft in best_feature_positions_bow.astype(int):
        bow_best_words.append(vec_bow.get_feature_names()[ft])
        
    tf_best_words = []
    for ft in best_feature_positions_tf.astype(int):
        tf_best_words.append(vec_bow.get_feature_names()[ft])
        
    tfidf_best_words = []
    for ft in best_feature_positions_tfidf.astype(int):
        tfidf_best_words.append(vec_tfidf.get_feature_names()[ft])
    
    # reverse-sort lists so they are in proper order
    bow_best_words.reverse()
    tf_best_words.reverse()
    tfidf_best_words.reverse()    
    
    models[i] = (bow_log, tf_log, tfidf_log, bow_best_words, tf_best_words, tfidf_best_words)
    
print 'Cell Runtime:', round(time.time() - cell_start), 'seconds'

Progression: Logistic Regression for dreamer  0
Progression: Logistic Regression for dreamer  1
Progression: Logistic Regression for dreamer  2


/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Progression: Logistic Regression for dreamer  3
Progression: Logistic Regression for dreamer  4
Progression: Logistic Regression for dreamer  5
Progression: Logistic Regression for dreamer  6
Progression: Logistic Regression for dreamer  7
Progression: Logistic Regression for dreamer  8
Progression: Logistic Regression for dreamer  9
Progression: Logistic Regression for dreamer  10
Progression: Logistic Regression for dreamer  11
Progression: Logistic Regression for dreamer  12
Progression: Logistic Regression for dreamer  13
Progression: Logistic Regression for dreamer  14
Progression: Logistic Regression for dreamer  15
Progression: Logistic Regression for dreamer  16
Progression: Logistic Regression for dreamer  17
Progression: Logistic Regression for dreamer  18
Progression: Logistic Regression for dreamer  19
Progression: Logistic Regression for dreamer  20
Progression: Logistic Regression for dreamer  21
Progression: Logistic Regression for dreamer  22
Progression: Logistic Regre

## Most Predictive Words

Now that we have a separate model for each dreamer, we can pull out the most predictive words from each model. This shows, for a given dreamer, which words are most predictive of their dreams as opposed to someone else's dream.

In [12]:
#==================================================================
# Cell purpse: Show the most predictive words from models per dreamer
#              BOW
#              TF (term frequence)
#              TFIDF
#==================================================================

for key, (bow_mod, tf_mod, tfidf_mod, bow_best_words, tf_best_words, tfidf_best_words) in models.iteritems():
    print 'W266ID='+str(key)
    print 'Most Predictive BOW', bow_best_words
    print 'Most Predictive TF', tf_best_words    
    print 'Most Predictive TF-IDF', tfidf_best_words, '\n'

W266ID=0
Most Predictive BOW ['dreamed', 'setting', 'characters', 'pregnant', 'border']
Most Predictive TF ['setting', 'thoughts', 'dreamed', 'border', 'whom']
Most Predictive TF-IDF [u'dreamed', u'td', u'setting', u'li', u'suddenly'] 

W266ID=1
Most Predictive BOW ['re', 'here', 'somebody', 'somewhere', 'rather']
Most Predictive TF ['re', 'here', 'somebody', 'or', 'something']
Most Predictive TF-IDF [u've', u'somebody', u'maybe', u'nice', u'think'] 

W266ID=2
Most Predictive BOW ['pancakes', 'meet', 'campus', 'quarter', 'children']
Most Predictive TF ['children', 'baby', 'boyfriend', 'then', 'people']
Most Predictive TF-IDF [u'boyfriend', u'pancakes', u'housemates', u'quarter', u'preschool'] 

W266ID=3
Most Predictive BOW ['model', 'husband', 'picture', 'poison', 'hometown']
Most Predictive TF ['town', 'husband', 'hometown', 'am', 'son']
Most Predictive TF-IDF [u'hometown', u'husband', u'town', u'local', u'picture'] 

W266ID=4
Most Predictive BOW ['scary', 'neat', 'nightmare', 'nightm

## Predict Each Dreamer

We can use our models to predict which person a given dream came from. If we take a dream from our test set and run all 41 models on that dream, we will get probabilities of the dream coming from that person. We can then take the highest probability and make that our prediction for who the dream came from. 

In [13]:
#==================================================================
# Cell Purpse: Do Prection based on the Individual Dreamer Model
#              Using BOW, TF, TFIDF
#==================================================================

# predict dreamer for each test dream
vec_bow_test_data   = vec_bow.transform(test_data)
vec_tf_test_data    = vec_tf.transform(vec_bow_test_data)
vec_tfidf_test_data = vec_tfidf.transform(test_data)

bow_preds = []
tf_preds = []
tfidf_preds = []
for dream in range(len(test_labels)):
    highest_prob_bow = 0
    highest_prob_tf = 0 
    highest_prob_tfidf = 0    
    
    # predicted probability of the correct label for each model
    for key, (bow_mod, tf_mod, tfidf_mod, bow_best_words, tf_best_words, tfidf_best_words) in models.iteritems():
        # BOW        
        prob_correct_bow =  bow_mod.predict_proba(vec_bow_test_data[dream])[0][1]
        if prob_correct_bow > highest_prob_bow:
            highest_prob_bow = prob_correct_bow
            bow_pred = key
        
        # TF 
        prob_correct_tf =  tf_mod.predict_proba(vec_tf_test_data[dream])[0][1]
        if prob_correct_tf > highest_prob_tf:
            highest_prob_tf = prob_correct_tf
            tf_pred = key     
        
        # TF-IDF
        prob_correct_tfidf =  tfidf_mod.predict_proba(vec_tfidf_test_data[dream])[0][1]
        if prob_correct_tfidf > highest_prob_tfidf:
            highest_prob_tfidf = prob_correct_tfidf
            tfidf_pred = key                 
    
    bow_preds.append(bow_pred)
    tf_preds.append(tf_pred)    
    tfidf_preds.append(tfidf_pred)    
    
print "BOW accuracy:", sum(bow_preds == df['W266ID'][20800:]) / len(bow_preds)
print "TF accuracy:", sum(tf_preds == df['W266ID'][20800:]) / len(tf_preds)
print "TF-IDF accuracy:", sum(tfidf_preds == df['W266ID'][20800:]) / len(tfidf_preds)

/anaconda/lib/python2.7/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


BOW accuracy: 0.814230769231
TF accuracy: 0.843269230769
TF-IDF accuracy: 0.842692307692


# Score 40 models separately

Scoring the models separately gives around 50% accuracy for all models. I tihnk the reason for that if becasue, for example, Dreamer1's model might be good at classifying Dreamer1's dreams, it will do poorly on everybody else's dreams and so the overall accuracy is only 50%. For everyone who isn't Dreamer1, the model has a 50/50 chance to get it right and since the overwhelming majority of dreamers aren't dreamer1, then the overall accuracy is roughly 50/50.

*The code below is commented out, because this step is not useful (as described above).*

In [14]:
# vec_bow_test_data = vec_bow.transform(test_data)
# vec_tfidf_test_data = vec_tfidf.transform(test_data)

# for key, (bow_model, tfidf_model, bow_score, tfidf_score, w_bow, w_tfidf) in models.iteritems():
#     bow_scores = []
#     tfidf_scores = []    
#     for i in range(0,41):
#         # split data
#         train_data, train_labels, dev_data, dev_labels, test_data, test_labels = split_data(df, W266ID=i)
        
#         # score model
#         f1_bow = metrics.f1_score(test_labels, bow_model.predict(vec_bow_test_data), average='macro')
#         bow_scores.append(f1_bow)

#         f1_tfidf = metrics.f1_score(test_labels, tfidf_model.predict(vec_tfidf_test_data), average='macro')
#         tfidf_scores.append(f1_tfidf)        

# #     print "Dreamer=%s \tAccuracy=%3.2f" % (key, score)
#     print "Dreamer=%s \tAccuracy=%3.2f \t%3.2f" % (key, np.mean(bow_scores), np.mean(tfidf_scores))

In [15]:
# df[df["W266ID"]!=0][['W266ID', 'Name']].sort_values(by= ["W266ID"]).drop_duplicates()


# Topic Modeling

## With Noun-Only Reduction

In [16]:
# #==================================================================
# # Cell Purpose: Set up the df to do Topic Modeling 
# #               Split the dream documents into tokens
# #==================================================================

# dreams = list(df['Dream'])

# # Split the documents into tokens.
# for idx in range(len(dreams)):
#     dreams[idx] = dreams[idx].lower()  # Convert to lowercase.
#     dreams[idx] = nltk.tokenize.word_tokenize(dreams[idx])  # Split into words.
#     dreams[idx] = nltk.pos_tag(dreams[idx])  # tag with PoS
#     dreams[idx] = [token for token, tag in dreams[idx] if tag.startswith('N')]   # only keep nouns 
    
# # Remove numbers, but not words that contain numbers.
# dreams = [[token for token in dream if not token.isdigit()] for dream in dreams]

# # Remove words that are only one or two characters.
# dreams = [[token for token in dream if len(token) > 2] for dream in dreams]

# # Lemmatize the dreams.
# lmtzr = WordNetLemmatizer()
# dreams = [[lmtzr.lemmatize(token) for token in dream] for dream in dreams]

In [17]:
# # Remove rare and common tokens.
# # Create a dictionary representation of the dreams.
# dictionary = Dictionary(dreams)

# # Filter out words that occur less than 5 dreams, or more than 60% of the dreams.
# dictionary.filter_extremes(no_below=5, no_above=0.6)

# # Vectorize data.

# # Bag-of-words representation of the documents.
# corpus = [dictionary.doc2bow(dream) for dream in dreams]

# print('Number of unique tokens: %d' % len(dictionary))
# print('Number of dreams: %d' % len(corpus))

__BELOW IS FROM: https://markroxor.github.io/gensim/static/notebooks/lda_training_tips.html__

__HELPS EXPLAIN THE PARAMETERS__

## Training

*We are ready to train the LDA model. We will first discuss how to set some of the training parameters.*

*First of all, the elephant in the room: how many topics do I need? There is really no easy answer for this, it will depend on both your data and your application. I have used 10 topics here because I wanted to have a few topics that I could interpret and "label", and because that turned out to give me reasonably good results. You might not need to interpret all your topics, so you could use a large number of topics, for example 100.*

*The `chunksize` controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory. I've set chunksize = 2000, which is more than the amount of documents, so I process all the data in one go. Chunksize can however influence the quality of the model, as discussed in Hoffman and co-authors [2], but the difference was not substantial in this case.*

*`passes` controls how often we train the model on the entire corpus. Another word for passes might be "epochs". iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. It is important to set the number of "passes" and "iterations" high enough.*

*I suggest the following way to choose iterations and passes. First, enable logging (as described in many Gensim tutorials), and set eval_every = 1 in LdaModel. When training the model look for a line in the log that looks something like this: *

    `2016-06-21 15:40:06,753 - gensim.models.ldamodel - DEBUG - 68/1566 documents converged within 400 iterations`

*If you set `passes` = 20 you will see this line 20 times. Make sure that by the final passes, most of the documents have converged. So you want to choose both passes and iterations to be high enough for this to happen.*

*We set `alpha = 'auto'` and `eta = 'auto'`. Again this is somewhat technical, but essentially we are automatically learning two parameters in the model that we usually would have to specify explicitly. *

In [18]:
# # Train LDA model.

# # Set training parameters.
# num_topics = 50
# chunksize = 20000
# passes = 30
# iterations = 400
# eval_every = None  # Don't evaluate model perplexity, takes too much time.

# # Make a index to word dictionary.
# temp = dictionary[0]  # This is only to "load" the dictionary.
# id2word = dictionary.id2token

# %time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
#                        alpha='auto', eta='auto', iterations=iterations, num_topics=num_topics, \
#                        passes=passes, eval_every=eval_every)

In [19]:
# top_topics = model.top_topics(corpus, topn=10)

# # Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
# avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
# print('Average topic coherence: %.4f.' % avg_topic_coherence)

# from pprint import pprint
# pprint(top_topics)

In [20]:

## FOR NOW WE DON'T WANT TO RUN DIFFERENT TOPICS FOR EACH DREAMER

# ### Run Topic Modeling on each dreamer separately
# import time

# # dict to store topic models for each dreamer
# topic_models = dict()

# for dreamer_id in range(1,41):
#     loop_start_time = time.time()
    
#     dreams = list(df[df['W266ID'] == dreamer_id]['Dream'])

#     # Set training parameters.
#     num_topics = 10
#     chunksize = 3000
#     passes = 20
#     iterations = 400
#     eval_every = None  # Don't evaluate model perplexity, takes too much time.

#     # Make a index to word dictionary.
#     temp = dictionary[0]  # This is only to "load" the dictionary.
#     id2word = dictionary.id2token

#     mode_start_time = time.time()
#     model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
#                            alpha='auto', eta='auto', iterations=iterations, num_topics=num_topics, \
#                            passes=passes, eval_every=eval_every)
#     print 'DreamerID', str(dreamer_id) + ': Total cell time=' + str(time.time() - loop_start_time), \
#             '\t Model build time=' + str(time.time() - mode_start_time)
    
#     topic_models[dreamer_id] = (model, corpus, dictionary)
    


In [59]:
# # show top topics for dreamerID #1
# for key, (mod, corp, dic) in topic_models.iteritems():
#     if key == 1:
#         print "\nDreamer", key
#         pprint(mod.top_topics(corpus = corp))

## With Full Corpus
Topic modelling is done by transformations (in gensim)
LDA must be trained before use using a BOW

In [60]:
#==================================================================
# Cell Purpose:  Use all 26000 documents to create Corpus
#                Train a LDA model on full corpus
#==================================================================
dreams = list(df['Dream'])

# Split the documents into tokens.
for idx in range(len(dreams)):
    dreams[idx] = dreams[idx].lower()  # Convert to lowercase.
    dreams[idx] = nltk.tokenize.word_tokenize(dreams[idx])  # Split into words.
    
print "Progress Update: Finished splitting documents"
# Remove numbers, but not words that contain numbers.
dreams = [[token for token in dream if not token.isdigit()] for dream in dreams]

print "Progress Update: removed numbers"
# Remove words that are only one or two characters.
dreams = [[token for token in dream if len(token) > 2] for dream in dreams]

# Remove other words that are artifacts of data
# remove /d, cellpadding, =, etc.

print "Progress Update: Removed short words "
# Lemmatize the dreams.
lmtzr = WordNetLemmatizer()
dreams = [[lmtzr.lemmatize(token) for token in dream] for dream in dreams]

Progress Update: Finished splitting documents
Progress Update: removed numbers
Progress Update: Removed short words 


In [61]:
DEBUG_VERBOSITY_HIGH = False
if DEBUG_VERBOSITY_HIGH:
    print len(dreams)
    for i in range(0,100):
        print len(dreams[i])

In [62]:
#==================================================================
# Cell Purpose: Create a dictionary representation of the dreams
#               Corpus is a Bag-of-words representation
#==================================================================

## Remove rare and common tokens.
dictionary = Dictionary(dreams)

# Filter out words that occur less than 5 dreams, 
# Experimented with removing more than 60% of the dreams.
# But this was shown to have no effect (screen out more than 60% of dreams)
dictionary.filter_extremes(no_below=5, no_above=0.5)

#----------------
# Vectorize data.
#----------------
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(dream) for dream in dreams]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of dreams: %d' % len(corpus))


Number of unique tokens: 13862
Number of dreams: 26000


In [63]:
print(corpus[1])
print len(dictionary.token2id)

[(232, 1), (727, 2), (1054, 1), (1179, 2), (1263, 1), (1528, 1), (1642, 1), (1681, 1), (2062, 2), (2435, 1), (2777, 1), (5575, 2), (5754, 1), (6465, 5), (6691, 1), (6968, 1), (7271, 1), (7397, 1), (7472, 1), (8286, 1), (8427, 1), (8487, 1), (8869, 2), (9593, 1), (9786, 2), (9891, 3), (10085, 1), (10252, 1), (10497, 1), (11315, 1), (12176, 2), (12219, 3), (12420, 1), (12542, 1), (13195, 1), (13426, 1)]
13862


In [64]:
#==================================================================
# Cell Purpse:  Create a dictionary representation of the dreams.
#==============================================================
# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token


In [65]:
#==================================================================
# Cell Purpse: Load  previously trained LDA model 
#              or else train LDA Model
#================================================================== 
CREATE_FULL_CORPUS_LDA_MODEL = False
MAX_TOPICS = True
LDA_MODEL_200_TOPIC_PATH_FILE = './model/topics200/DreamLda200.model'
# LDA_MODEL_100_TOPIC_PATH_FILE = './model/topics100/DreamLda100.model'
LDA_MODEL_50_TOPIC_PATH_FILE  = './model/topics50/DreamLda.model'

if CREATE_FULL_CORPUS_LDA_MODEL:
    # Set training parameters.
    if MAX_TOPICS:
        num_topics = 200
    else:
        num_topics = 50
    chunksize = 26000/10
    passes = 10
    iterations = 400
    eval_every = None

    %time dreamerLdaModel = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                           alpha='auto', eta='auto', iterations=iterations, num_topics=num_topics, \
                           passes=passes, eval_every=eval_every)
    if MAX_TOPICS:
        dreamerLdaModel.save(LDA_MODEL_200_TOPIC_PATH_FILE)
    else: 
        dreamerLdaModel.save(LDA_MODEL_50_TOPIC_PATH_FILE)
else: 
    if MAX_TOPICS:
        # Model with 200 Topics
        dreamerLdaModel = LdaModel.load(LDA_MODEL_200_TOPIC_PATH_FILE)
    else:
        # Model with 50 Topics
        dreamerLdaModel = LdaModel.load(LDA_MODEL_50_TOPIC_PATH_FILE)

In [66]:
top_topics = dreamerLdaModel.top_topics(corpus, topn=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / len(dreamerLdaModel.get_topics())
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -16.7119.
[([(0.042795285017861708, u'not'),
   (0.023145344661499073, u'have'),
   (0.022983324152422026, u'more'),
   (0.020107495999448704, u'than'),
   (0.018575112110018839, u'are'),
   (0.017555468889634954, u'this'),
   (0.01727830329026623, u'for'),
   (0.016264237139442986, u'know'),
   (0.015084273307661709, u'people'),
   (0.014874713026676577, u'something'),
   (0.013737556888346365, u'seems'),
   (0.012495329008543665, u'ha'),
   (0.011566440756693422, u'can'),
   (0.010660663171711863, u'think'),
   (0.010389396006430157, u'about'),
   (0.010378595219282553, u'all'),
   (0.0095552822800807381, u'though'),
   (0.0091024055291368516, u'what'),
   (0.0088095271205246399, u'doe'),
   (0.0086460788680038803, u'want')],
  -10.065473579052345),
 ([(0.025894399828842497, u"'re"),
   (0.023012061302491487, u'this'),
   (0.018807336298975456, u'get'),
   (0.018271174754536119, u'some'),
   (0.017921280034214813, u'here'),
   (0.017109112964021587, u"'ve"),


   (0.041834960545301982, u'left'),
   (0.038519533318145609, u'right'),
   (0.033334440101650542, u'direction'),
   (0.032324925816740784, u'side'),
   (0.024364670854789505, u'walking'),
   (0.021194025951725964, u'far'),
   (0.019534872328967147, u'across'),
   (0.019237394962292341, u'ahead'),
   (0.018235054917989279, u'entrance'),
   (0.017775704901540422, u'along'),
   (0.016672666843703464, u'manager'),
   (0.015226108989456246, u'standing'),
   (0.014735138703899411, u'distance'),
   (0.013974744564348222, u'partner'),
   (0.012711382627538056, u'enter'),
   (0.012704430678472087, u'which'),
   (0.01094867571528583, u'both'),
   (0.010463150747206556, u'wide')],
  -12.981450084148999),
 ([(0.55129616490288857, u'room'),
   (0.065640721156405138, u'into'),
   (0.05947598845323275, u'kitchen'),
   (0.029876263854598632, u'floor'),
   (0.023593327313682553, u'wall'),
   (0.018106668011398767, u'another'),
   (0.017744042399622916, u'hall'),
   (0.015269189851326231, u'naked'),
  

   (0.03071573818601258, u'awake'),
   (0.029247600212026718, u'frightened'),
   (0.016628181718892614, u'unhappy'),
   (0.016253614769462212, u'myself'),
   (0.015906959270134174, u'because'),
   (0.015903049155628679, u'been'),
   (0.015120833589622679, u'le'),
   (0.014807353581335239, u'from'),
   (0.013734206053239238, u'successful'),
   (0.013727105923645819, u'recently'),
   (0.012593417973326013, u'plug'),
   (0.012321503450626109, u'negative')],
  -15.195504821901412),
 ([(0.44116188744096108, u'she'),
   (0.35036383628939688, u'her'),
   (0.023215709586553356, u'woman'),
   (0.0079103366071904095, u'tell'),
   (0.007850055191668483, u'daughter'),
   (0.0074035288635957751, u'snake'),
   (0.0058757478207458521, u'see'),
   (0.0056682731831486166, u'not'),
   (0.005645143863936046, u'back'),
   (0.0053532966074639901, u'talking'),
   (0.0052786986453238639, u'doe'),
   (0.0052595814291139069, u'arm'),
   (0.0052483528894050127, u'for'),
   (0.0051561193623981191, u'about'),
   

   (0.015741610517794283, u'all'),
   (0.013868953093114733, u'elephant'),
   (0.013458460216276731, u'when'),
   (0.011570297650456937, u'school'),
   (0.0086930071950555386, u'then'),
   (0.0084002402964153972, u'know'),
   (0.0071107594682934296, u'school.'),
   (0.0070350396587871645, u'big'),
   (0.0057923186841156837, u'people'),
   (0.0052812050003531721, u'zipper')],
  -16.159320836316518),
 ([(0.19639470527694816, u'dad'),
   (0.17370418332055948, u'brother'),
   (0.069009322301384055, u'john'),
   (0.063993322954202292, u'ticket'),
   (0.044056133599865967, u'mom'),
   (0.023451061252554018, u'david'),
   (0.022666747155647465, u'ezra'),
   (0.019890192317933368, u'jim'),
   (0.01988297806741212, u'about'),
   (0.019287443290121161, u'heap'),
   (0.018385996629797223, u'rover'),
   (0.012912000023518562, u'ryan'),
   (0.011874790872050808, u'email'),
   (0.011459069327263975, u'diary'),
   (0.011028240953243822, u'silas'),
   (0.010880436157042957, u'going'),
   (0.0106379368

   (0.019356791325710075, u'eaten'),
   (0.019065372196585818, u'bat'),
   (0.017816903206083902, u'rusty'),
   (0.016403700777318894, u'shark'),
   (0.015475806664820766, u'third'),
   (0.010528331357721145, u'disgusting'),
   (0.009968683296982625, u'typewriter'),
   (0.009126255380502794, u'undressed')],
  -17.072444849786482),
 ([(0.19620653254401538, u'change'),
   (0.06599968305428193, u'changed'),
   (0.0502172265294396, u'page'),
   (0.046947852663738812, u'normal'),
   (0.042017383162551623, u'bigger'),
   (0.041519427625293641, u'medium'),
   (0.030787681125297645, u'model'),
   (0.023559210797368099, u'length'),
   (0.023326672525446759, u'bleacher'),
   (0.023282839994288509, u'reaching'),
   (0.015310412790504675, u'straighten'),
   (0.013421871040918501, u'oldest'),
   (0.012840489127386124, u'youngest'),
   (0.01270855064172181, u'ms.'),
   (0.012011152218587125, u'remodeled'),
   (0.011470451632831094, u'sized'),
   (0.011193370028840369, u'greg'),
   (0.010887599346488

   (0.034936428044707458, u'serve'),
   (0.03210230624156693, u'celine'),
   (0.032091272741586296, u'flashlight'),
   (0.031307692492164724, u'witness'),
   (0.026056696362559979, u'fist'),
   (0.024596785365399349, u'dion'),
   (0.024282698294036852, u'enters'),
   (0.022442359555457914, u'beneath'),
   (0.021648195934014083, u'interior'),
   (0.021154009502528576, u'as'),
   (0.020811419137090965, u'belligerent'),
   (0.0172103183083822, u'locate'),
   (0.016863129466097453, u'verbally'),
   (0.014039279915425045, u'elise')],
  -17.758679640964434),
 ([(0.52230680806589957, u'group'),
   (0.057962652274639877, u'gym'),
   (0.05592332992186691, u'people'),
   (0.045237434437978272, u'drinking'),
   (0.035554804664112497, u'tour'),
   (0.019816935642770331, u'leather'),
   (0.019775414389637404, u'encounter'),
   (0.017604994608097616, u'diamond'),
   (0.016960864121133144, u'comfortable'),
   (0.016813257610444183, u'spent'),
   (0.01675275561671901, u'including'),
   (0.015832252812

   (0.010504858025633333, u'slot'),
   (0.0096703935412807852, u'answered'),
   (0.0095194317931419983, u'toddler'),
   (0.0087080017065067714, u'talking')],
  -18.529990119049735),
 ([(0.15386321736268033, u'grandma'),
   (0.068618482745770495, u'jane'),
   (0.06561475672748325, u'brake'),
   (0.054269663131904393, u'attic'),
   (0.033205363504008031, u'maid'),
   (0.032199660560994235, u'gear'),
   (0.031214480439161772, u'birth'),
   (0.030768374285335531, u'teasing'),
   (0.025275377193657824, u'formal'),
   (0.024403414220798201, u'california'),
   (0.024188793406803799, u'pissed'),
   (0.023445347313271386, u'expected'),
   (0.02337559449268508, u'license'),
   (0.02262811198720658, u'maternal'),
   (0.021878731467026721, u'treat'),
   (0.014509842522141771, u'unconscious'),
   (0.013880528266629076, u'tender'),
   (0.013213735368550137, u'safely'),
   (0.013132330239343931, u'strangely'),
   (0.012499736293592942, u'explore')],
  -18.533054555083794),
 ([(0.2275367242948278, u'm

   (0.016345481560882516, u'enclosed'),
   (0.015431338534505335, u'ejaculate'),
   (0.014359534887186097, u'senior'),
   (0.013758993749255847, u'mailbox'),
   (0.012367857902220461, u'darker')],
  -19.116483451126065),
 ([(0.1743753876507452, u'moving'),
   (0.048087686910307181, u'move'),
   (0.040676791975227937, u'view'),
   (0.036151445520690308, u'awhile'),
   (0.032636762592790018, u'wine'),
   (0.02991690197108814, u'complete'),
   (0.029392346685078903, u'secretary'),
   (0.025022335718781555, u'load'),
   (0.023813961943850919, u'property'),
   (0.022426905560136719, u'england'),
   (0.017444683307063689, u'steering'),
   (0.015520106822602624, u'poking'),
   (0.015333099201862635, u'shack'),
   (0.014170582189692081, u'inner'),
   (0.012540314221748153, u'shock'),
   (0.011445983832839065, u'slight'),
   (0.011039983159596107, u'organize'),
   (0.010907992380657242, u'barrier'),
   (0.01079174071750251, u'kicking'),
   (0.010651692280058916, u'causing')],
  -19.134604754431

In [67]:
# Print a few of the most 
print dreamerLdaModel.print_topics(-1)

[(0, u'0.166*"hit" + 0.091*"throw" + 0.074*"catch" + 0.048*"tunnel" + 0.037*"throwing" + 0.036*"hitting" + 0.032*"baseball" + 0.018*"fast" + 0.017*"trapped" + 0.017*"run"'), (1, u'0.642*"water" + 0.099*"lake" + 0.025*"into" + 0.023*"spider" + 0.020*"under" + 0.017*"clear" + 0.016*"sunny" + 0.012*"rising" + 0.010*"dear" + 0.009*"grass"'), (2, u'0.069*"ten" + 0.045*"shortly" + 0.040*"straight" + 0.039*"teenage" + 0.037*"surprise" + 0.035*"linda" + 0.034*"reunion" + 0.020*"poker" + 0.020*"jessica" + 0.019*"agency"'), (3, u'0.072*"matt" + 0.071*"blond" + 0.057*"grown" + 0.042*"nude" + 0.041*"____" + 0.038*"serving" + 0.031*"appearance" + 0.031*"sister-in-law" + 0.031*"assigned" + 0.028*"chick"'), (4, u'0.104*"st." + 0.074*"comment" + 0.050*"wanting" + 0.040*"larry" + 0.039*"whale" + 0.036*"decoration" + 0.034*"accepted" + 0.033*"christmas" + 0.030*"willie" + 0.029*"statement"'), (5, u'0.128*"bob" + 0.079*"paid" + 0.074*"display" + 0.057*"won" + 0.056*"die" + 0.054*"boot" + 0.047*"article" 

In [68]:
#==================================================================
# Cell Purpse: 1)Take a dreamer from the Collection
#              2) Create a BOW for each dream
#              3) Create the LDA_Vector 
#              4) Compute the top topics per dreaml
#================================================================== 

# Take Dreamer i and transform the dreams into a bow
# cell_start = time.time()
DEBUG_VERBOSITY_HIGH = False


text = "The one at the Meads's house, where it's bigger inside than out; there's a European village just inside, with a cobblestone street and a Pied-Piper sort of man with curly hair, he can do things like juggle - I go up the back stairs [there aren't any in the real house] and then down the other side [since there's a second set, immediately] then down a short empty hallway that turns a corner, where I find a tiny room...a young woman with shoulder-length blonde hair in a pageboy is there, cooking at a stove that almost fills the room...she's nice to me. Now outside, I'm waiting for my aunt to pick me up - she arrives in a little round convertible and we go for a drive, not very far - we cross a little bridge over a creek, then double back and she drops me off at the house again. Inside (?) I sit with a couple of people, playing with a string of blue balloons."
#bow_vector = dictionary.doc2bow(dream)
bow_vector = dictionary.doc2bow(nltk.tokenize.word_tokenize(text))
bow_vector2 = dictionary.doc2bow(dreams[1])
lda_vector = dreamerLdaModel[bow_vector2]   # topic probability distribution
print lda_vector
print dreamerLdaModel.print_topic(max(lda_vector, key=lambda item: item[1])[0])


if DEBUG_VERBOSITY_HIGH:
    print len(bow_vector2)
    #print bow_vector2
    print [(dictionary[id], count) for id, count in bow_vector2]
    print "\n"
    print len(bow_vector)
    print [(dictionary[id], count) for id, count in bow_vector]
    

[(3, 0.035291615380589468), (7, 0.018558945088643173), (15, 0.036078433177916633), (22, 0.035331078453446296), (24, 0.017801297566707699), (39, 0.017820273131724473), (67, 0.035324803573490833), (68, 0.01825297135638183), (75, 0.10165433844259322), (80, 0.033102266603749472), (91, 0.020465408848421716), (93, 0.047199991575402382), (94, 0.017885495261806519), (98, 0.017942913541684834), (101, 0.017815957784904846), (104, 0.017870802061267288), (116, 0.018007794710063765), (119, 0.035662369210159806), (122, 0.083126771915936606), (132, 0.013290260653198839), (134, 0.03742240546154809), (136, 0.088368357155032801), (164, 0.053505995057580323), (173, 0.017836567864160546), (182, 0.018172905260143417), (183, 0.017979085503251706), (186, 0.019266020966255019), (196, 0.019328903305715892)]
0.054*"had" + 0.043*"for" + 0.041*"get" + 0.040*"could" + 0.030*"out" + 0.026*"would" + 0.025*"time" + 0.018*"when" + 0.018*"did" + 0.017*"going"


In [69]:
lda_vector = dreamerLdaModel[bow_vector]
# print lda_vector 
print dreamerLdaModel.print_topic(max(lda_vector, key=lambda item: item[1])[0])

0.008*"mushroom" + 0.007*"emerald" + 0.006*"arnold" + 0.005*"sobbing" + 0.005*"speeding" + 0.005*"prefer" + 0.005*"examined" + 0.005*"cary" + 0.004*"tee" + 0.004*"morgan"


# Author Topic Modeling 
Reference: https://nbviewer.jupyter.org/github/rare-technologies/gensim/blob/develop/docs/notebooks/atmodel_tutorial.ipynb

In [70]:
#==================================================================
# Cell Purpse: Create author2doc
#              mapping of authors to dreamIDs/docIDs
#================================================================== 
author2doc = dict()

for docID in range(len(df)):
    authorID = df['W266ID'][docID]
    if not author2doc.get(authorID):
        # This is a new author.
        author2doc[authorID] = []
    
    # Add document IDs to author
    author2doc[authorID].extend([docID])        

In [74]:
#==================================================================
# Cell Purpse: Load previously trained Author Topic Model 
#              or else train Author Topic Model 
#================================================================== 
from gensim.models import AuthorTopicModel

CREATE_FULL_CORPUS_AUTHOR_MODEL = False

if CREATE_FULL_CORPUS_AUTHOR_MODEL:
    
    model_list = []
    for i in [50,100,150,200]:
        model_path = './model/authorModels/AuthorLDA'+str(i)+'.model'
        %time AuthorModel = AuthorTopicModel(corpus=corpus, num_topics=i, id2word=dictionary.id2token, \
                                    author2doc=author2doc, chunksize=2600, passes=100, eval_every=None, \
                                    iterations=400, alpha='auto', eta='auto')
        top_topics = AuthorModel.top_topics(corpus)
        tc = sum([t[1] for t in top_topics])
        model_list.append((AuthorModel, tc))
        AuthorModel.save(model_path)
    
else: 
    model_list = []
    for i in [50,100,150,200]:
        model_path = './model/authorModels/AuthorLDA'+str(i)+'.model'
        AuthorModel = AuthorTopicModel.load(model_path)
        model_list.append(AuthorModel)
        
model, tc = max(model_list, key=lambda x: x[1])
print('Best model topic coherence: %.3e' %tc)        

CPU times: user 49min 40s, sys: 3min 36s, total: 53min 16s
Wall time: 53min 40s
CPU times: user 1h 17min 12s, sys: 13min 55s, total: 1h 31min 8s
Wall time: 1h 36min 28s
CPU times: user 1h 37min 7s, sys: 25min 25s, total: 2h 2min 33s
Wall time: 2h 3min 46s
CPU times: user 1h 53min 5s, sys: 38min 19s, total: 2h 31min 24s
Wall time: 2h 31min 28s
Best model topic coherence: -5.313e+01


In [75]:
# Example topic. It doesn't look useful/helpful
model.show_topic(0)

[(u'she', 0.010827252168250243),
 (u'for', 0.010727197814510101),
 (u'her', 0.0078852487951760631),
 (u'some', 0.0073896227107565328),
 (u'about', 0.0067097079718064796),
 (u'from', 0.006561214187664562),
 (u'had', 0.0064892822297569937),
 (u'one', 0.006426499907705701),
 (u'they', 0.0064044532750068385),
 (u'then', 0.0062362526909947478)]

In [149]:
for topic in model.show_topics(num_topics=10):
    print "Topic:", topic[0]
    words = ''
    for word, prob in model.show_topic(topic[0]):
        words += word + ' '
    print 'Words: ' + words 
    print

Topic: 11
Words: she for her some about from had one they then 

Topic: 2
Words: she for her some about from had one they then 

Topic: 18
Words: she for her some about from had one they then 

Topic: 45
Words: she for her some about from had one they then 

Topic: 22
Words: she for her some about from had one they then 

Topic: 14
Words: are have friend her she ha from get people for 

Topic: 43
Words: say are see man him woman her then feel have 

Topic: 19
Words: had were dream then friend remember when this not for 

Topic: 13
Words: had were they one them did for she about said 

Topic: 37
Words: she then out like her they them one room get 



In [122]:
def show_author(name):
    print('\nDreamerID:%d' % name)
    print('Topics:')
    pprint([(topic[0], topic[1]) for topic in model[name]])

In [123]:
show_author(1)


DreamerID:1
Topics:
[(13, 0.077470633554747315),
 (14, 0.050693620757122054),
 (19, 0.048697299104955259),
 (37, 0.11245753439696292),
 (38, 0.070772838140230068),
 (41, 0.56483675739314265),
 (43, 0.055941617724067735),
 (49, 0.014277067575665138)]


In [124]:
from gensim.similarities import MatrixSimilarity

# Generate a similarity object for the transformed corpus.
index = MatrixSimilarity(model[list(model.id2author.values())])

In [144]:
# Get similarities to some author.
def get_table(author_id, top_n=10, smallest_author=1):
    '''
    Get table with similarities, author IDs, and author sizes.
    Return `top_n` authors as a dataframe.   
    '''
    
    # Get similarities.
    sims = index[model[author_id]]

    # Arrange author names, similarities, and author sizes in a list of tuples.
    table = []
    for elem in enumerate(sims):
        author_name = model.id2author[elem[0]]
        sim = elem[1]
        author_size = len(model.author2doc[author_name])
        if author_size >= smallest_author:
            table.append((author_name, sim, author_size))
            
    # Make dataframe and retrieve top authors.
    df = pd.DataFrame(table, columns=['AuthorID', 'Cosine Distance', 'Size'])
    df = df.sort_values('Cosine Distance', ascending=False)[:top_n]
    
    return df

In [145]:
# authors most similar to Dreamer #1
get_table(1)

,AuthorID,Cosine Distance,Size
1,1,1.000000,422
37,37,0.911805,33
38,38,0.857504,27
7,7,0.728969,75
6,6,0.675339,100
35,35,0.638258,16
3,3,0.629270,212
5,5,0.615527,53
4,4,0.563336,4254
9,9,0.502758,166
